# Soportes Tickets

In [ ]:
# Paquetes
from datetime import datetime
import os
from dotenv import load_dotenv
import pandas as pd
from plataforma_web.app import create_app
from plataforma_web.extensions import db
from plataforma_web.blueprints.oficinas.models import Oficina
from plataforma_web.blueprints.soportes_categorias.models import SoporteCategoria
from plataforma_web.blueprints.soportes_tickets.models import SoporteTicket
from plataforma_web.blueprints.usuarios.models import Usuario

In [ ]:
# Inicializar
load_dotenv()  # Take environment variables from .env
app = create_app()
db.app = app

In [ ]:
# Rango de tiempos
creado_desde = datetime(year=2022,month=2,day=14,hour=0,minute=0,second=0)
creado_hasta = datetime(year=2022,month=2,day=18,hour=23,minute=59,second=59)

In [ ]:
# Comenzar consulta de tickets
soportes_tickets = SoporteTicket.query.\
    filter(SoporteTicket.creado >= creado_desde).\
    filter(SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    filter(SoporteTicket.estatus == "A")

In [ ]:
# Total de tickets por oficina
for oficina in Oficina.query.filter_by(estatus="A").all():
    soportes_tickets_por_oficina = soportes_tickets.join(Usuario).filter(Usuario.oficina_id == oficina.id)
    if soportes_tickets_por_oficina.count() > 0:
        print("En", oficina.clave, "hay", soportes_tickets_por_oficina.count(), "tickets CERRADOS")

In [ ]:
# Total de tickets en una categoria
categoria = SoporteCategoria.query.get(68)  # Firma Electronica (68)
tickets_en_una_categoria = soportes_tickets.filter(SoporteTicket.soporte_categoria == categoria)
print("Hay", tickets_en_una_categoria.count(), "tickets CERRADOS en la categoria", categoria.nombre)

In [ ]:
# Total de tickets en la categoria por oficina
for oficina in Oficina.query.filter_by(estatus="A").all():
    tickets_por_oficina = tickets_en_una_categoria.join(Usuario).filter(Usuario.oficina_id == oficina.id)
    if tickets_por_oficina.count() > 0:
        print("En", oficina.clave, "hay", tickets_por_oficina.count(), "tickets CERRADOS en la categoria", soporte_categoria.nombre)

In [ ]:
# Elaborar un dataframe con las categorias en renglones y las oficinas en columnas
data = {}
categorias = SoporteCategoria.query.filter_by(estatus="A").order_by(SoporteCategoria.nombre).all()
tickets = SoporteTicket.query.\
    filter(SoporteTicket.creado >= creado_desde).\
    filter(SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    filter(SoporteTicket.estatus == "A")
for oficina in Oficina.query.filter_by(estatus="A").order_by(Oficina.clave.asc()).all():
    tickets_por_oficina = tickets.join(Usuario).filter(Usuario.oficina_id == oficina.id)
    renglon = {}
    for categoria in categorias:
        tickets_por_categoria = tickets_por_oficina.filter(SoporteTicket.soporte_categoria == categoria)
        renglon[categoria.nombre] = tickets_por_categoria.count()
    data[oficina.clave] = renglon
df = pd.DataFrame(data)

In [ ]:
df_limpio = df.loc[:, (df.sum(axis=0) > 0)]
df_limpio

In [ ]:
# Guardar como archivo CSV
df_limpio.to_csv("soportes_tickets_cerrados_por_categoria.csv", index=True, encoding="utf-8")